# Comptek

## Инициализация

In [ ]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


import re
import catalog.runner
from catalog.models import *


class Runner(catalog.runner.Runner):

    name  = 'Comptek'
    alias = 'comptek'
    url = {
        'start'    : 'http://comptek.ru/',
        'login'    : 'http://comptek.ru/personal/auth.xhtml',
        'price'    : 'http://comptek.ru/',
        'filter'   : 'catalog/',
        'unfilter' : 'item/',
        'base'     : 'http://comptek.ru'}

    def __init__(self):

        super().__init__()

        self.stock    = self.take_stock('stock', 'склад', 3, 10)
        self.transit  = self.take_stock('transit', 'транзит', 10, 60)
        self.on_order = self.take_stock('on-order', 'на заказ', 40, 80)

        self.count = {'product': 0, 'party': 0}

    def run(self):

        import time

        payload = {
            'login'    : self.updater.login,
            'password' : self.updater.password}
        if self.login(payload):
            print('Авторизован.')
        else:
            print('Не удалось авторизоваться')
            return False

        # Заходим на начальную страницу каталога
        tree = self.load_html(self.url['price'])

        # TODO Проходим по всем ссылкам
        urls = []
        for u in tree.xpath('//a/@href'):
            if self.url['base'] not in u:
                u = self.url['base'] + urls[i]

            
            if u not in urls:
                urls.append(u)

        i = 0
        while i < len(urls):

            # Сслыка на категорию
            if self.url['filter'] in urls[i]:
                print('Загружаю:', urls[i])

                vendor = Vendor.objects.get_by_key(updater = self.updater, key = url.split('/')[4])
                print('Vendor: {}.'.format(vendor))

                if vendor:
                    tree = self.load_html(url)
                    print("Загружена: {}.".format(url))

                    for u in tree.xpath('//a/@href'):
                        if not u in urls:
                            urls.append(u)

                    # Парсим таблицу с товарами
#                    self.parse(tree, vendor)

                    # Ждем, чтобы не получить отбой сервера
                    time.sleep(1)

                else:
                    print("Пропущена: {}.".format(url))

            else:
                url = self.url['base'] + urls[i]
                print("Пропущена: {}.".format(url))

            i += 1

        # Чистим партии
        Party.objects.clear(stock = self.stock, time = self.start_time)
        Party.objects.clear(stock = self.transit, time = self.start_time)
        Party.objects.clear(stock = self.on_order, time = self.start_time)

        self.log()

s = Runner()

In [ ]:
s.run()